# Resampling Methods - Lab

## Introduction

Now that you have some preliminary background on bootstrapping, jackknife, and permutation tests, its time to practice those skills by coding them into functions. You'll then apply these tests to a hypothesis test and compare the results to a parametric t-test.

## Objectives

In this lab you will: 

* Create functions that perform resampling techniques and use them on datasets

## Bootstrap sampling


Bootstrap sampling works by combining two distinct samples into a universal set and generating random samples from this combined sample space in order to compare these random splits to the two original samples. The idea is to see if the difference between the two **original** samples is statistically significant. If similar differences can be observed through the random generation of samples, then the observed differences are not actually significant.


Write a function to perform bootstrap sampling. The function should take in two samples A and B. The two samples need not be the same size. From this, create a universal sample by combining A and B. Then, create a resampled universal sample of the same size using random sampling with replacement. Finally, split this randomly generated universal set into two samples which are the same size as the original samples, A and B. The function should return these resampled samples.

Example:

```python

A = [1,2,3]
B = [2,2,5,6]

Universal_Set = [1,2,2,2,3,5,6]
Resampled_Universal_Set = [6, 2, 3, 2, 1, 1, 2] # Could be different (randomly generated with replacement)

Resampled_A = [6,2,3]
Resampled_B = [2,1,1,2]
```

In [1]:
import random

def bootstrap(A, B):
    # Combine the samples
    universal_set = A + B
    
    # Resample with replacement
    resampled_universal_set = [random.choice(universal_set) for _ in range(len(universal_set))]
    
    # Split the resampled set
    resampled_A = resampled_universal_set[:len(A)]
    resampled_B = resampled_universal_set[len(A):]
    
    return resampled_A, resampled_B


## Jackknife 

Write a function that creates additional samples by removing one element at a time. The function should do this for each of the `n` items in the original sample, returning `n` samples, each with `n-1` members.

In [2]:
def jack1(sample):
    """This function should take in a list of n observations and return n lists
    each with one member (presumably the nth) removed."""
    
    # Initialize an empty list to store the jackknife samples
    jackknife_samples = []
    
    # Loop through each element in the sample
    for i in range(len(sample)):
        # Create a new sample by excluding the current element
        jackknife_sample = sample[:i] + sample[i+1:]
        # Append the new sample to the list of jackknife samples
        jackknife_samples.append(jackknife_sample)
    
    return jackknife_samples

sample = [1, 2, 3, 4]
jackknife_samples = jack1(sample)
print(jackknife_samples)



[[2, 3, 4], [1, 3, 4], [1, 2, 4], [1, 2, 3]]


## Permutation testing

Define a function that generates all possible, equally sized, two set splits of two sets A and B. Sets A and B need not be the same size, but all of the generated two set splits should be of equal size. For example, if we had a set with 5 members and a set with 7 members, the function would return all possible 5-7 ordered splits of the 12 items.

> Note that these are actually combinations! However, as noted previously, permutation tests really investigate possible regroupings of the data observations, so calculating combinations is a more efficient approach!


Here's a more in depth example:

```python
A = [1, 2, 2]
B = [1, 3]
combT(A, B) 
[([1,2,2], [1,3]),
 ([1,2,3], [1,2]),
 ([1,1,2], [2,3]),
 ([1,1,3], [2,2]),
 ([2,2,3], [1,1])]
               
```  

These are all the possible 3-2 member splits of the 5 elements: 1, 1, 2, 2, 3. 

In [3]:
from itertools import combinations

def combT(A, B):
    # Combine the sets
    universal_set = A + B
    
    # Get the size of the sets
    size_A = len(A)
    size_B = len(B)
    
    # Initialize a list to store the splits
    splits = []
    
    # Generate all combinations of the universal set of size_A
    for combo in combinations(universal_set, size_A):
        # Convert combo to a list
        combo_A = list(combo)
        # Generate the complementary set for B
        combo_B = [item for item in universal_set if item not in combo_A or combo_A.remove(item)]
        # Append the split to the list
        splits.append((combo_A, combo_B))
    
    return splits


## Permutation testing in Practice
Let's further investigate the scenario proposed in the previous lesson. Below are two samples A and B. The samples are mock data for the blood pressure of sample patients. The research study is looking to validate whether there is a statistical difference in the blood pressure of these two groups using a 5% significance level.  First, calculate the mean blood pressure of each of the two samples. Then, calculate the difference of these means. From there, use your `combT()` function, defined above, to generate all the possible combinations of the entire sample data into A-B splits of equivalent sizes as the original sets. For each of these combinations, calculate the mean blood pressure of the two groups and record the difference between these sample means. The full collection of the difference in means between these generated samples will serve as the denominator to calculate the p-value associated with the difference between the original sample means.

For example, in our small handwritten example above:

$\mu_a = \frac{1+2+2}{3} = \frac{5}{3}$  
and  
$\mu_b = \frac{1+3}{2} = \frac{4}{2} = 2$  

Giving us

$\mu_a - \mu_b = \frac{5}{3} - 2 = -\frac{1}{3}$

In comparison, for our various combinations we have:

([1,2,2], [1,3]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = -\frac{1}{3}$  
([1,2,3], [1,2]):  $\mu_a - \mu_b = 2 - \frac{3}{2} = \frac{1}{2}$  
([1,2,1], [2,3]):  $\mu_a - \mu_b = \frac{4}{3} - \frac{5}{3} = -\frac{1}{2}$  
([1,1,3], [2,2]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = -\frac{1}{3}$  
([2,2,3], [1,1]):  $\mu_a - \mu_b = \frac{7}{3} - 1 = \frac{4}{3}$  

A standard hypothesis test for this scenario might be:

$H_0: \mu_a = \mu_b$  
$H_1: \mu_a < \mu_b$  
  
Thus comparing our sample difference to the differences of our possible combinations, we look at the number of experiments from our combinations space that were the same or greater than our sample statistic, divided by the total number of combinations. In this case, 4 out of 5 of the combination cases produced the same or greater differences in the two sample means. This value .8 is a strong indication that we cannot refute the null hypothesis for this instance.

In [ ]:
a = [109.6927759 , 120.27296943, 103.54012038, 114.16555857,
       122.93336175, 110.9271756 , 114.77443758, 116.34159338,
       112.66413025, 118.30562665, 132.31196515, 117.99000948]
b = [123.98967482, 141.11969004, 117.00293412, 121.6419775 ,
       123.2703033 , 123.76944385, 105.95249634, 114.87114479,
       130.6878082 , 140.60768727, 121.95433026, 123.11996767,
       129.93260914, 121.01049611]

In [4]:
# Your code here
# ⏰ Expect your code to take several minutes to run

import numpy as np
from itertools import combinations

# Define the combT function
def combT(A, B):
    # Combine the sets
    universal_set = A + B
    
    # Get the size of the sets
    size_A = len(A)
    size_B = len(B)
    
    # Initialize a list to store the splits
    splits = []
    
    # Generate all combinations of the universal set of size_A
    for combo in combinations(universal_set, size_A):
        # Convert combo to a list
        combo_A = list(combo)
        # Generate the complementary set for B
        combo_B = [item for item in universal_set if item not in combo_A or combo_A.remove(item)]
        # Append the split to the list
        splits.append((combo_A, combo_B))
    
    return splits

# Given data
a = [109.6927759 , 120.27296943, 103.54012038, 114.16555857,
     122.93336175, 110.9271756 , 114.77443758, 116.34159338,
     112.66413025, 118.30562665, 132.31196515, 117.99000948]

b = [123.98967482, 141.11969004, 117.00293412, 121.6419775 ,
     123.2703033 , 123.76944385, 105.95249634, 114.87114479,
     130.6878082 , 140.60768727, 121.95433026, 123.11996767,
     129.93260914, 121.01049611]

# Calculate the means of the original samples
mean_a = np.mean(a)
mean_b = np.mean(b)

# Calculate the difference of the means
original_mean_diff = mean_a - mean_b

# Generate all possible combinations
splits = combT(a, b)

# Calculate the mean differences for each combination
mean_diffs = []
for split in splits:
    mean_a_split = np.mean(split[0])
    mean_b_split = np.mean(split[1])
    mean_diffs.append(mean_a_split - mean_b_split)

# Calculate the p-value
mean_diffs = np.array(mean_diffs)
p_value = np.sum(mean_diffs >= original_mean_diff) / len(mean_diffs)

# Print the results
print(f"Original mean difference: {original_mean_diff}")
print(f"P-value: {p_value}")


C:\Users\svijayaraghavan\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\svijayaraghavan\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Original mean difference: -8.049348947857169
P-value: 0.0


## T-test revisited

The parametric statistical test equivalent to our permutation test above would be a t-test of the two groups. Perform a t-test on the same data above in order to calculate the p-value. How does this compare to the above results?

In [ ]:
# Your code here
import numpy as np
from itertools import combinations
from scipy.stats import ttest_ind

# Define the combT function
def combT(A, B):
    # Combine the sets
    universal_set = A + B
    
    # Get the size of the sets
    size_A = len(A)
    size_B = len(B)
    
    # Initialize a list to store the splits
    splits = []
    
    # Generate all combinations of the universal set of size_A
    for combo in combinations(universal_set, size_A):
        # Convert combo to a list
        combo_A = list(combo)
        # Generate the complementary set for B
        combo_B = [item for item in universal_set if item not in combo_A or combo_A.remove(item)]
        # Append the split to the list
        splits.append((combo_A, combo_B))
    
    return splits

# Given data
a = [109.6927759 , 120.27296943, 103.54012038, 114.16555857,
     122.93336175, 110.9271756 , 114.77443758, 116.34159338,
     112.66413025, 118.30562665, 132.31196515, 117.99000948]

b = [123.98967482, 141.11969004, 117.00293412, 121.6419775 ,
     123.2703033 , 123.76944385, 105.95249634, 114.87114479,
     130.6878082 , 140.60768727, 121.95433026, 123.11996767,
     129.93260914, 121.01049611]

# Calculate the means of the original samples
mean_a = np.mean(a)
mean_b = np.mean(b)

# Calculate the difference of the means
original_mean_diff = mean_a - mean_b

# Generate all possible combinations
splits = combT(a, b)

# Calculate the mean differences for each combination
mean_diffs = []
for split in splits:
    mean_a_split = np.mean(split[0])
    mean_b_split = np.mean(split[1])
    mean_diffs.append(mean_a_split - mean_b_split)

# Calculate the p-value
mean_diffs = np.array(mean_diffs)
p_value_permutation = np.sum(mean_diffs >= original_mean_diff) / len(mean_diffs)

# Perform the t-test
t_stat, p_value_ttest = ttest_ind(a, b, equal_var=False)

# Print the results
print(f"Original mean difference: {original_mean_diff}")
print(f"Permutation test p-value: {p_value_permutation}")
print(f"T-test p-value: {p_value_ttest}")


C:\Users\svijayaraghavan\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\svijayaraghavan\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## Bootstrap applied

Use your code above to apply the bootstrap technique to this hypothesis testing scenario. Here's a pseudo-code outline for how to do this:

1. Compute the difference between the sample means of A and B
2. Initialize a counter for the number of times the difference of the means of resampled samples is greater then or equal to the difference of the means of the original samples
3. Repeat the following process 10,000 times:
    1. Use the bootstrap sampling function you used above to create new resampled versions of A and B 
    2. Compute the difference between the means of these resampled samples 
    3. If the difference between the means of the resampled samples is greater then or equal to the original difference, add 1 the counter you created in step 2
4. Compute the ratio between the counter and the number of simulations (10,000) that you performed
    > This ratio is the percentage of simulations in which the difference of sample means was greater than the original difference

In [ ]:
# Your code here
import numpy as np
from itertools import combinations
from scipy.stats import ttest_ind

# Define the combT function
def combT(A, B):
    # Combine the sets
    universal_set = A + B
    
    # Get the size of the sets
    size_A = len(A)
    size_B = len(B)
    
    # Initialize a list to store the splits
    splits = []
    
    # Generate all combinations of the universal set of size_A
    for combo in combinations(universal_set, size_A):
        # Convert combo to a list
        combo_A = list(combo)
        # Generate the complementary set for B
        combo_B = [item for item in universal_set if item not in combo_A or combo_A.remove(item)]
        # Append the split to the list
        splits.append((combo_A, combo_B))
    
    return splits

# Given data
a = [109.6927759 , 120.27296943, 103.54012038, 114.16555857,
     122.93336175, 110.9271756 , 114.77443758, 116.34159338,
     112.66413025, 118.30562665, 132.31196515, 117.99000948]

b = [123.98967482, 141.11969004, 117.00293412, 121.6419775 ,
     123.2703033 , 123.76944385, 105.95249634, 114.87114479,
     130.6878082 , 140.60768727, 121.95433026, 123.11996767,
     129.93260914, 121.01049611]

# Calculate the means of the original samples
mean_a = np.mean(a)
mean_b = np.mean(b)

# Calculate the difference of the means
original_mean_diff = mean_a - mean_b

# Generate all possible combinations
splits = combT(a, b)

# Calculate the mean differences for each combination
mean_diffs = []
for split in splits:
    mean_a_split = np.mean(split[0])
    mean_b_split = np.mean(split[1])
    mean_diffs.append(mean_a_split - mean_b_split)

# Calculate the p-value for permutation test
mean_diffs = np.array(mean_diffs)
p_value_permutation = np.sum(mean_diffs >= original_mean_diff) / len(mean_diffs)

# Perform the t-test
t_stat, p_value_ttest = ttest_ind(a, b, equal_var=False)

# Perform bootstrap resampling
# Initialize the counter
count = 0

# Number of bootstrap samples
n_iterations = 10000

# Perform bootstrap resampling
for _ in range(n_iterations):
    # Resample A and B with replacement
    resample_a = np.random.choice(a, size=len(a), replace=True)
    resample_b = np.random.choice(b, size=len(b), replace=True)
    
    # Calculate the means of the resampled samples
    mean_resample_a = np.mean(resample_a)
    mean_resample_b = np.mean(resample_b)
    
    # Calculate the difference of the means of the resampled samples
    resample_mean_diff = mean_resample_a - mean_resample_b
    
    # Increment the counter if the resampled mean difference is greater than or equal to the original mean difference
    if resample_mean_diff >= original_mean_diff:
        count += 1

# Calculate the p-value for bootstrap
p_value_bootstrap = count / n_iterations

# Print the results
print(f"Original mean difference: {original_mean_diff}")
print(f"Permutation test p-value: {p_value_permutation}")
print(f"T-test p-value: {p_value_ttest}")
print(f"Bootstrap p-value: {p_value_bootstrap}")


## Summary

Well done! In this lab, you practice coding modern statistical resampling techniques of the 20th century! You also started to compare these non-parametric methods to other parametric methods such as the t-test that we previously discussed.